In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import ribopy
from ribopy import Ribo
from ribopy.core.get_gadgets import get_region_boundaries, get_reference_names
from ribopy_functions import get_cds_range_lookup, apris_human_alias, get_psite_offset, get_sequence
import gzip
import pickle

In [2]:
ribo_path   = "all.ribo"
ribo_object = Ribo(ribo_path, alias = ribopy.api.alias.apris_human_alias)
reference_file_path = "appris_mouse_v2_selected.fa.gz"

In [3]:
with gzip.open('HSC_vehicle_vs_GMP_vehicle_negForwarded.pkl.gz', 'rb') as f:
    coverage_dict = pickle.load(f)

In [4]:
cds_range = get_cds_range_lookup(ribo_object)
sequence = get_sequence(ribo_object, reference_file_path, True)

In [5]:
sequence

{'Xkr4-201': 'GCGGCGGCGGGCGAGCGGGCGCTGGAGTAGGAGCTGGGGAGCGGCGCGGCCGGGGAAGGAAGCCAGGGCGAGGCGAGGAGGTGGCGGGAGGAGGAGACAGCAGGGACAGGTGTCAGATAAAGGAGTGCTCTCCTCCGCTGCCGAGGCATCATGGCCGCTAAGTCAGACGGGAGGCTGAAGATGAAGAAGAGCAGCGACGTGGCGTTCACCCCGCTGCAGAACTCGGACAATTCGGGCTCTGTGCAAGGACTGGCTCCAGGCTTGCCGTCGGGGTCCGGAGCCGAGGACACGGAGGCGGCCGGAGGCGGCTGCTGCCCGGACGGCGGTGGCTGCTCGCGCTGCTGCTGCTGCTGCGCGGGGAGCGGCGGCTCGGCGGGCTCGGGCGGCTCGGGCGGCGGCGGCCGGGGCAGCGGGGCGGGCTCTGCGGCGCTGTGCCTGCGCCTGGGCAGGGAGCAGCGGCGTTACTCGCTGTGGGACTGCCTCTGGATCCTGGCCGCCGTGGCCGTGTACTTCGCGGATGTGGGAACGGACATCTGGCTCGCGGTGGACTACTACCTGCGTGGCCAGCGCTGGTGGTTTGGGCTCACCCTCTTCTTCGTGGTGCTGGGCTCCCTTTCTGTGCAAGTGTTCAGCTTCCGCTGGTTTGTGCATGATTTCAGCACCGAGGACAGCTCCACGACCACCACCTCCAGCTGCCAGCAGCCTGGAGCAGATTGCAAGACGGTGGTCAGCAGTGGGTCTGCAGCCGGGGAAGGCGAGGTTCGTCCTTCCACGCCGCAGAGGCAAGCATCCAACGCCAGCAAGAGCAACATCGCCGCCACCAACAGCGGCAGCAACAGCAACGGGGCCACCCGGACCAGCGGCAAACACAGGTCTGCGTCCTGCTCCTTTTGCATCTGGCTCCTGCAGTCACTCATCCACATCTTGCAGCTTGGGCAAATCTGGAGGTATTTGCACACAATATACTTAGGTATCCGGAGCCGGCA

In [14]:

from itertools import islice

transcript_np = ribo_object.transcript_names
fasta = FastaFile(reference_file_path)
fasta_dict = {e.header: e.sequence for e in fasta}
dict(islice(fasta_dict.items(), 5))
# fasta_dict = {e.header: e.sequence for e in fasta}

{'ENSMUST00000070533.4|ENSMUSG00000051951.5|OTTMUSG00000026353.2|OTTMUST00000065166.1|Xkr4-201|Xkr4|3634|UTR5:1-150|CDS:151-2094|UTR3:2095-3634|': 'GCGGCGGCGGGCGAGCGGGCGCTGGAGTAGGAGCTGGGGAGCGGCGCGGCCGGGGAAGGAAGCCAGGGCGAGGCGAGGAGGTGGCGGGAGGAGGAGACAGCAGGGACAGGTGTCAGATAAAGGAGTGCTCTCCTCCGCTGCCGAGGCATCATGGCCGCTAAGTCAGACGGGAGGCTGAAGATGAAGAAGAGCAGCGACGTGGCGTTCACCCCGCTGCAGAACTCGGACAATTCGGGCTCTGTGCAAGGACTGGCTCCAGGCTTGCCGTCGGGGTCCGGAGCCGAGGACACGGAGGCGGCCGGAGGCGGCTGCTGCCCGGACGGCGGTGGCTGCTCGCGCTGCTGCTGCTGCTGCGCGGGGAGCGGCGGCTCGGCGGGCTCGGGCGGCTCGGGCGGCGGCGGCCGGGGCAGCGGGGCGGGCTCTGCGGCGCTGTGCCTGCGCCTGGGCAGGGAGCAGCGGCGTTACTCGCTGTGGGACTGCCTCTGGATCCTGGCCGCCGTGGCCGTGTACTTCGCGGATGTGGGAACGGACATCTGGCTCGCGGTGGACTACTACCTGCGTGGCCAGCGCTGGTGGTTTGGGCTCACCCTCTTCTTCGTGGTGCTGGGCTCCCTTTCTGTGCAAGTGTTCAGCTTCCGCTGGTTTGTGCATGATTTCAGCACCGAGGACAGCTCCACGACCACCACCTCCAGCTGCCAGCAGCCTGGAGCAGATTGCAAGACGGTGGTCAGCAGTGGGTCTGCAGCCGGGGAAGGCGAGGTTCGTCCTTCCACGCCGCAGAGGCAAGCATCCAACGCCAGCAAGAGCAACATCGCCGCCACCAACAGCGGCAGCAACAGCAACGGGGCCACC

In [6]:
# coverage_dict['GMP_vehicle_A']['Adgre4-201'] + coverage_dict['GMP_vehicle_B']['Adgre4-201']

array([0, 0, 0, ..., 0, 0, 0], dtype=uint16)

# archive

In [4]:
file_path = "output/genes_list/HSC_vehicle_vs_GMP_vehicle_negForwarded.txt"
with open(file_path, 'r') as file:
    gene_list = [line.strip() for line in file]

In [5]:
def apris_human_alias(x):
    return x.split("|")[4]

In [38]:
def process_transcript(transcript, exp, min_len, max_len, alias, cds_range, offset, ribo_object):
    start, stop = cds_range[transcript]
    coverages = []
    for i in range(min_len, max_len + 1): 
        if offset[i] <= start:
            coverage = ribo_object.get_coverage(experiment=exp, range_lower=i, range_upper=i, alias=alias)\
                       [transcript][start - offset[i] : stop - offset[i]]
            coverages.append(coverage)
        else:
            coverage = ribo_object.get_coverage(experiment=exp, range_lower=i, range_upper=i, alias=alias)\
                       [transcript][: stop - offset[i]]
            coverage = np.concatenate((np.zeros(offset[i] - start), coverage))
            coverages.append(coverage)
            
    coverage = sum(coverages, np.zeros_like(coverages[0]))
    
    return transcript, coverage

In [6]:
def process_transcript(transcript, exp, min_len, max_len, alias, cds_range, offset, ribo_object):
    # Create a mapping of read lengths to offsets
    length_groups = {}
    for length in range(min_len, max_len + 1):
        if offset[length] not in length_groups:
            length_groups[offset[length]] = []
        length_groups[offset[length]].append(length)

    # Initialize an empty list for storing coverage data
    coverages = []

    # Loop over the grouped offsets and process coverage for all read lengths with the same offset
    for group_offset, lengths in length_groups.items():
        # Sort lengths to identify continuous ranges
        lengths.sort()
        
        # Break the lengths into continuous segments
        continuous_ranges = []
        current_range = [lengths[0]]

        for i in range(1, len(lengths)):
            if lengths[i] == lengths[i - 1] + 1:  # Check if the current length is continuous
                current_range.append(lengths[i])
            else:
                continuous_ranges.append(current_range)
                current_range = [lengths[i]]
        
        # Don't forget to add the last range
        continuous_ranges.append(current_range)

        # Fetch coverage for each continuous range
        for length_range in continuous_ranges:
            min_len_in_range = min(length_range)
            max_len_in_range = max(length_range)
            
            # Get coverage data for the current range
            coverage_data = ribo_object.get_coverage(experiment=exp, range_lower=min_len_in_range, 
                                                     range_upper=max_len_in_range, alias=alias)
            
            # Process each length in the continuous range
            for length in length_range:
                start, stop = cds_range[transcript]
                offset_value = group_offset

                # Check if coverage needs to be adjusted based on the offset
                if length <= start:
                    coverage = coverage_data[transcript][start - offset_value : stop - offset_value]
                else:
                    coverage = coverage_data[transcript][: stop - offset_value]
                    coverage = np.concatenate((np.zeros(offset_value - start), coverage))
                
                coverages.append(coverage)

    # Sum up the coverages for all read lengths
    final_coverage = sum(coverages, np.zeros_like(coverages[0]))
    
    return transcript, final_coverage

In [ ]:
exp = "GMP_vehicle_A"
min_len = 20
max_len = 38
alias = True
cds_range = get_cds_range_lookup(ribo_object)
offset = get_psite_offset(ribo_object, exp, min_len, max_len)

coverage_dict = {}
for gene in gene_list:
    transcript = apris_human_alias(next(str(name) for name in ribo_object.transcript_names if gene in name))
    coverage = process_transcript(transcript, exp, min_len, max_len, alias, cds_range, offset, ribo_object)
    coverage_dict[transcript] = coverage

/var/folders/jd/fjgmspbj6zs0pqckr9dv96f40000gn/T/ipykernel_82366/929680436.py:50: RuntimeWarning: overflow encountered in scalar subtract
  coverage = np.concatenate((np.zeros(offset_value - start), coverage))


In [59]:
length_groups = {}
for length in range(min_len, max_len + 1):
    if offset[length] not in length_groups:
        length_groups[offset[length]] = []
    length_groups[offset[length]].append(length)

for group_offset, lengths in length_groups.items():
    # Sort lengths to identify continuous ranges
    lengths.sort()
    
    # Break the lengths into continuous segments
    continuous_ranges = []
    current_range = [lengths[0]]

    for i in range(1, len(lengths)):
        if lengths[i] == lengths[i - 1] + 1:  # Check if the current length is continuous
            current_range.append(lengths[i])
        else:
            continuous_ranges.append(current_range)
            current_range = [lengths[i]]
    
    # Don't forget to add the last range
    continuous_ranges.append(current_range)
    print(continuous_ranges)

[[20, 21, 22], [24, 25, 26, 27, 28], [31, 32, 33, 34, 35, 36, 37, 38]]
[[23]]
[[29, 30]]


In [54]:
length_groups

{14: [20, 21, 22, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38],
 11: [23],
 13: [29, 30]}